In [2]:
import gzip
import json
import pandas as pd
import geopandas as gpd
import os
from tqdm import tqdm

In [6]:
# 📁 Dossier contenant les fichiers cadastraux convertis en .parquet
data_dir = "data_parquet"
excel_file = "parcelle_keep.xlsx"

# 📥 Charger la liste des parcelles à conserver
parcels_to_keep = pd.read_excel(excel_file, engine='openpyxl')
parcels_to_keep = parcels_to_keep.dropna(subset=["commune", "section", "numero"])
parcels_to_keep = parcels_to_keep[parcels_to_keep['numero'] != 0]

# Normalisation des champs
parcels_to_keep['commune'] = parcels_to_keep['commune'].astype(str).str.strip()
parcels_to_keep['section'] = parcels_to_keep['section'].astype(str).str.strip().str.upper()
parcels_to_keep['numero'] = parcels_to_keep['numero'].astype(str).str.strip().str.zfill(4)
parcels_to_keep['prefixe'] = parcels_to_keep['prefixe'].fillna(0).astype(float).astype(int).astype(str).str.zfill(3)

# 📦 Listes temporaires pour concaténation
parcels_chunks = []
buildings_chunks = []

# 🔁 Parcourir les fichiers Parquet
for filename in tqdm(os.listdir(data_dir), desc="Départements traités"):
    if filename.endswith("-parcelles.parquet"):
        code_dept = filename.split("-")[1].split(".")[0]
        parcels_path = os.path.join(data_dir, f"cadastre-{code_dept}-parcelles.parquet")
        buildings_path = os.path.join(data_dir, f"cadastre-{code_dept}-batiments.parquet")

        parcels_gdf = gpd.read_parquet(parcels_path)
        buildings_gdf = gpd.read_parquet(buildings_path)

        # 🧹 Nettoyage (Cadastre)
        for col in ['commune', 'section', 'numero']:
            parcels_gdf[col] = parcels_gdf[col].astype(str).str.strip()
        parcels_gdf['section'] = parcels_gdf['section'].str.upper()
        parcels_gdf['numero'] = parcels_gdf['numero'].astype(str).str.strip().str.zfill(4)
        if 'prefixe' not in parcels_gdf.columns:
            parcels_gdf['prefixe'] = "000"
        else:
            parcels_gdf['prefixe'] = parcels_gdf['prefixe'].fillna("000").astype(str).str.strip().str.zfill(3)

        # 🎯 Filtrer uniquement les parcelles du département
        communes_dept = parcels_gdf['commune'].unique()
        subset_to_keep = parcels_to_keep[parcels_to_keep['commune'].isin(communes_dept)]

        # 🔍 Filtrage exact
        filtered_parcels = parcels_gdf.merge(
            subset_to_keep,
            on=['commune', 'section', 'numero', 'prefixe'],
            how='inner'
        )

        print(f"📂 Département {code_dept} : {len(filtered_parcels)} parcelles filtrées")

        if filtered_parcels.empty:
            continue

        # 🧭 Intersection avec les bâtiments
        join_intersects = gpd.sjoin(
            buildings_gdf,
            filtered_parcels,
            how='inner',
            predicate='intersects'
        ).reset_index(drop=True)

        # Calcul du ratio de recouvrement
        parcel_geometries = filtered_parcels.geometry.loc[join_intersects["index_right"]].reset_index(drop=True)
        join_intersects["intersection_area"] = join_intersects.geometry.intersection(parcel_geometries).area
        join_intersects["building_area"] = join_intersects.geometry.area
        join_intersects["coverage_ratio"] = join_intersects["intersection_area"] / join_intersects["building_area"]
        filtered_buildings = join_intersects[join_intersects["coverage_ratio"] >= 0.5].copy()

        # ➕ Ajouter aux listes
        parcels_chunks.append(filtered_parcels)
        buildings_chunks.append(filtered_buildings)

# 🧱 Concaténation finale
all_filtered_parcels = pd.concat(parcels_chunks, ignore_index=True)
all_filtered_buildings = pd.concat(buildings_chunks, ignore_index=True)

all_filtered_parcels.set_crs("EPSG:4326", inplace=True)
all_filtered_buildings.set_crs("EPSG:4326", inplace=True)

# 💾 Export
all_filtered_parcels.to_file("parcelles_filtrées.geojson", driver="GeoJSON")
all_filtered_buildings.to_file("parcelles_filtrées.geojson", driver="GeoJSON")
all_filtered_parcels[['commune', 'section', 'numero', 'prefixe']].to_excel("parcelles_trouvées.xlsx", index=False)

# Après concaténation finale
merged_all = parcels_to_keep.merge(
    all_filtered_parcels,
    on=['commune', 'section', 'numero', 'prefixe'],
    how='left',
    indicator=True,
    suffixes=('','_right')
)

# Parcelles absentes dans les résultats
cols_to_export = ['Code site', 'commune', 'section', 'numero', 'prefixe']
missing = merged_all[merged_all['_merge'] == 'left_only'][cols_to_export]
missing.to_excel("parcelles_non_trouvées.xlsx", index=False)

print(f"📤 {len(missing)} parcelles non retrouvées exportées dans 'parcelles_non_trouvées.xlsx'")

# 📊 Statistiques
print(f"📍 Base de sites : {len(parcels_to_keep['Code site'].unique())}")
trouvees_par_code_site = merged_all[merged_all['_merge'] == 'both']['Code site'].value_counts()
print(trouvees_par_code_site.head())


Départements traités:   0%|          | 0/2 [00:00<?, ?it/s]

📂 Département 12 : 2 parcelles filtrées


Départements traités: 100%|██████████| 2/2 [00:14<00:00,  7.36s/it]


📤 1248 parcelles non retrouvées exportées dans 'parcelles_non_trouvées.xlsx'
📍 Base de sites : 747
Code site
SITE_30524    2
Name: count, dtype: int64
